# Use pymatgen


In [1]:
from pathlib import Path

# Library with the functions that execute 
# Atoms and FEFF to generate scattering paths
import lib.atoms_feff as feff_runner   

from pymatgen.io.cif import CifParser, CifWriter
from pymatgen.io.feff.inputs import Atoms, Potential, Header


def create_feff_dir(feff_dir, feff_inp):
    feff_file = Path(feff_dir, feff_inp)
    feff_file.parent.mkdir(parents=True, exist_ok=True) 

def inp_from_cif(crystal_f, feff_dir, feff_inp,absorbing,c_radius):
    crystal_f = Path(cif_file)
    c_parser = CifParser(crystal_f)
    c_structure = c_parser.get_structures()[0]
    header_file = Path(feff_dir,"inp_header.txt")
    potentials_file = Path(feff_dir,"inp_potential.txt")
    atoms_file = Path(feff_dir,"inp_atoms.txt")
    
    # create inp contents from cif
    inp_header = Header.from_cif_file(cif_file, source=cif_file, comment="")
    inp_atoms = Atoms(c_structure,absorbing_atom=absorbing, radius=c_radius)
    inp_potential = Potential(c_structure,absorbing_atom=absorbing)
    
    # wirite individual inp content
    inp_header.write_file(header_file)
    inp_potential.write_file(potentials_file)
    inp_atoms.write_file(atoms_file)
    
    feff_file = Path(feff_dir, feff_inp)
    
    # join files into single inp file
    with open(feff_file, 'w' ) as result_file:
        # write header
        for line in open( header_file, 'r' ):
            result_file.write( line )
        # write RMAX
        result_file.write("\nRMAX      "+str(float(c_radius)) + "\n")
        # write potentials
        result_file.write("\n")
        for line in open( potentials_file, 'r' ):
            result_file.write( line )
        # write atoms
        result_file.write("\n")
        for line in open( atoms_file, 'r' ):
            result_file.write( line )
    
    
cif_file = "C:/harwell/PSDI_test/wf_data/pub_037/additional/1692395.cif"
crystal_f = Path(cif_file)
feff_dir = crystal_f.name[:-4]+"_feff"
feff_inp = crystal_f.name[:-4]+"_feff.inp"

create_feff_dir(feff_dir, feff_inp)

if crystal_f.name[-3:] != "inp":
    inp_from_cif(str(crystal_f), feff_dir, feff_inp, "Pt", 8)
    
    

In [2]:
feff_dir

'1692395_feff'

In [3]:
from larch.xafs.feffrunner import feff6l

feff6l(folder = feff_dir, feffinp = feff_inp)

 : ======== running Feff module C:\Users\scman1\AppData\Local\Programs\Python\Python39\lib\site-packages\larch\bin\win64\feff6l.exe ========
 : Feff 6L.02
 : comment: None given
 : Source:  C:/harwell/PSDI_test/wf_data/pub_037/additional/1692395.cif
 : Structure Summary:  Sn1 Pt3
 : Reduced formula:  SnPt3
 : space group: (Pm-3m), space number:  (221)
 : abc:  3.967000   3.967000   3.967000
 : angles: 90.000000  90.000000  90.000000
 : sites: 4
 : Calculating potentials and phases...
 : free atom potential and density for atom type    0
 : free atom potential and density for atom type    1
 : free atom potential and density for atom type    2
 : overlapped potential and density for unique potential    0
 : overlapped potential and density for unique potential    1
 : overlapped potential and density for unique potential    2
 : muffin tin radii and interstitial parameters
 : phase shifts for unique potential    0
 : phase shifts for unique potential    1
 : phase shifts for unique pote

 : 111     1.605    32.000     3   7.5426 neglected
 : 112     1.234    32.000     3   7.5426 neglected
 : 113     1.871    32.000     3   7.5426 neglected
 : 114     1.460    32.000     3   7.5426 neglected
 : 115     1.248    32.000     3   7.5426 neglected
 : 116     1.834    32.000     3   7.5426 neglected
 : 117     0.072    32.000     5   7.5937 neglected
 : 118     0.074    16.000     5   7.5937 neglected
 : 119     0.053    32.000     5   7.5937 neglected
 : 120     0.126    32.000     5   7.5937 neglected
 : 121     0.305    32.000     4   7.6637 neglected
 : 122     0.216    32.000     4   7.6637 neglected
 : 123     0.267    32.000     4   7.6637 neglected
 : 124     0.140    32.000     4   7.6637 neglected
 : 125     0.138    32.000     4   7.6637 neglected
 : 126     0.264    16.000     4   7.6637 neglected
 : 127     0.248    16.000     4   7.6637 neglected
 : 128     0.108    32.000     4   7.6637 neglected
 : 129     0.073    32.000     4   7.6637 neglected
 : 130     0

Attribute,Type
Feff8l_modules,tuple
run,method
folder,str
feffinp,str
verbose,bool
message_writer,NoneType
mpse,bool
resolved,NoneType
threshold,list
chargetransfer,list


In [4]:
#read an write cif files
from pymatgen.io.cif import CifParser, CifWriter
cif_file = "C:/harwell/PSDI_test/wf_data/pub_037/additional/1692395.cif"
cif_out = "1692395.cif"
parser = CifParser(cif_file)
structure = parser.get_structures()
cif_writer = CifWriter(parser.get_structures()[0])
cif_writer.write_file(cif_out)
parser2 = CifParser(cif_out)
structure2 = parser2.get_structures()[0]

In [5]:
from pymatgen.io.feff.inputs import Atoms, Potential, Header
out_inp = "1692395_feff.inp"
out_header = Header.from_cif_file(cif_file, source=cif_file, comment=parser.get_bibtex_string())
out_atoms = Atoms(structure2,absorbing_atom="Pt", radius=8)
out_potential = Potential(structure2,absorbing_atom="Pt")
#out_atoms.write_file(filename=out_inp)
#out_header.write_file(filename=out_inp)
#out_potential.write_file(filename=out_inp)


# Join the Header, potential and Header in a single file
#with open(out_inp, "a") as one_file:
out_header.write_file(out_inp)
out_potential.write_file(out_inp)
out_atoms.write_file(out_inp)


In [6]:
dir(parser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cif',
 '_frac_tolerance',
 '_get_structure',
 '_occupancy_tolerance',
 '_parse_symbol',
 '_sanitize_data',
 '_site_tolerance',
 '_unique_coords',
 'as_dict',
 'feature_flags',
 'from_string',
 'get_bibtex_string',
 'get_lattice',
 'get_lattice_no_exception',
 'get_magsymops',
 'get_structures',
 'get_symops',
 'has_errors',
 'parse_magmoms',
 'parse_oxi_states',
 'symmetry_operations',
 'warnings']

In [7]:
parser.as_dict()

{'183076-ICSD': {'_database_code_depnum_ccdc_archive': 'CCDC 1692395',
  '_citation_id': ['1'],
  '_citation_doi': ['10.1016/j.commatsci.2009.04.044'],
  '_citation_year': ['2009'],
  '_audit_update_record': ' 2018-02-28 deposited with the CCDC.\t2023-05-25 downloaded from the CCDC.',
  '_database_code_ICSD': '183076',
  '_chemical_name_systematic': 'Platinum Tin (3/1)',
  '_chemical_formula_sum': 'Pt3 Sn1',
  '_cell_length_a': '3.967',
  '_cell_length_b': '3.967',
  '_cell_length_c': '3.967',
  '_cell_angle_alpha': '90',
  '_cell_angle_beta': '90',
  '_cell_angle_gamma': '90',
  '_cell_volume': '62.43',
  '_cell_formula_units_Z': '1',
  '_symmetry_space_group_name_H-M': 'P m -3 m',
  '_symmetry_Int_Tables_number': '221',
  '_symmetry_cell_setting': 'cubic',
  '_symmetry_equiv_pos_site_id': ['1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9',
   '10',
   '11',
   '12',
   '13',
   '14',
   '15',
   '16',
   '17',
   '18',
   '19',
   '20',
   '21',
   '22',
   '23',
  

In [8]:
print(parser.get_bibtex_string())

@article{cifref0,
    year = "2009"
}



In [9]:
dir(parser)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cif',
 '_frac_tolerance',
 '_get_structure',
 '_occupancy_tolerance',
 '_parse_symbol',
 '_sanitize_data',
 '_site_tolerance',
 '_unique_coords',
 'as_dict',
 'feature_flags',
 'from_string',
 'get_bibtex_string',
 'get_lattice',
 'get_lattice_no_exception',
 'get_magsymops',
 'get_structures',
 'get_symops',
 'has_errors',
 'parse_magmoms',
 'parse_oxi_states',
 'symmetry_operations',
 'warnings']

In [10]:
structure 

[Structure Summary
 Lattice
     abc : 3.967 3.967 3.967
  angles : 90.0 90.0 90.0
  volume : 62.42903206300001
       A : 3.967 0.0 2.4290869261087753e-16
       B : 6.379422551267218e-16 3.967 2.4290869261087753e-16
       C : 0.0 0.0 3.967
     pbc : True True True
 PeriodicSite: Sn (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
 PeriodicSite: Pt (1.9835, 1.9835, 0.0000) [0.5000, 0.5000, 0.0000]
 PeriodicSite: Pt (1.9835, 0.0000, 1.9835) [0.5000, 0.0000, 0.5000]
 PeriodicSite: Pt (0.0000, 1.9835, 1.9835) [0.0000, 0.5000, 0.5000]]